# How to compute descriptors from Features extractors?

In [1]:
from geomfum.dataset import NotebooksDataset
from geomfum.descriptor.learned import FeatureExtractor, LearnedDescriptor
from geomfum.shape import TriangleMesh
import torch

[Load a mesh](00_load_mesh_from_file.ipynb).

In [2]:
dataset = NotebooksDataset()
mesh = TriangleMesh.from_file(dataset.get_filename("cat-00"))

INFO: Data has already been downloaded... using cached file ('/home/ubuntu/.geomfum/data/cat-00.off').
/home/ubuntu/giulio_vigano/geomfum_proj/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Set Laplace eigenbasis](./02_mesh_laplacian_spectrum.ipynb).

In [3]:
mesh.laplacian.find_spectrum(spectrum_size=10, set_as_basis=True)

mesh.basis

## DiffusionNet

In [4]:
descr = LearnedDescriptor(
    feature_extractor=FeatureExtractor.from_registry(which="diffusionnet")
)
with torch.no_grad():
    hsign = descr(mesh)

hsign = hsign

hsign.shape

(128, 7207)

# PointNet

In [5]:
descr = LearnedDescriptor(
    feature_extractor=FeatureExtractor.from_registry(which="pointnet")
)
with torch.no_grad():
    hsign = descr(mesh)

hsign = hsign

hsign.shape

(128, 7207)

# Descriptors as input

In [6]:
from geomfum.descriptor.spectral import HeatKernelSignature

descr = LearnedDescriptor(
    feature_extractor=FeatureExtractor.from_registry(
        which="diffusionnet",
        descriptor=HeatKernelSignature(n_domain=128),
        in_channels=128,
    )
)
with torch.no_grad():
    hsign = descr(mesh)

hsign = hsign

hsign.shape

(128, 7207)

# Pipeline

We can use learned features also concatenated with other descriptors in the pipeline

In [7]:
from geomfum.descriptor.pipeline import (
    DescriptorPipeline,
)
from geomfum.descriptor.spectral import HeatKernelSignature
from geomfum.descriptor.spectral import HeatKernelSignature
from geomfum.shape import TriangleMesh

In [8]:
steps = [
    HeatKernelSignature(n_domain=4),
    descr,
]

pipeline = DescriptorPipeline(steps)

hsign = pipeline.apply(mesh)
hsign.shape


(132, 7207)

# Saving and loading


In [9]:
descr = LearnedDescriptor()

descr.feature_extractor.save("./saved_model_test.pth")

In [10]:
descr2 = LearnedDescriptor()
descr2.feature_extractor.load_from_path("./saved_model_test.pth")

## Further reading

* [How to compute a functional map?](./07_functional_map.ipynb)

* [How to set landmarks?](./06_landmarks.ipynb)